# Generate database

The idea of this notebook is to test LLMs for structured data querying. For that, we'll create a local fake database for tests. The choice was to use sqlite for its praticity

In [1]:
import sqlite3
from faker import Faker
import pandas as pd

In [6]:


# Use Faker library to generate fake data
fake = Faker()

# Connect to SQLite database (creates a new database if not exists)
conn = sqlite3.connect('database.db')
cursor = conn.cursor()

# Create users table
cursor.execute('''
    CREATE TABLE IF NOT EXISTS users (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        username TEXT,
        email TEXT,
        reg_date DATE
    )
''')

# Generate and insert 100 rows of fake user data
for _ in range(100):
    username = fake.user_name()
    email = fake.email()
    reg_date = fake.date()
    
    cursor.execute('''
        INSERT INTO users (username, email, reg_date)
        VALUES (?, ?, ?)
    ''', (username, email, reg_date))

# Commit the changes and close the connection
conn.commit()
conn.close()

print("Sample database created successfully.")


Sample database created successfully.


In [2]:
conn = sqlite3.connect('database.db')
df = pd.read_sql_query('select * from users limit 5', conn)
df

,id,username,email,reg_date
0,1,catherine01,wilkersonjessica@example.com,1974-12-02
1,2,dsilva,cpotter@example.org,2009-05-13
2,3,santoskathryn,latasha53@example.net,2005-02-06
3,4,harrisbruce,christophernorris@example.com,2006-07-25
4,5,keithayala,maria39@example.net,2013-08-22


# Querying 
To query a SQLite database using Langchain and LLMs (Language Model Models), we will follow these steps.
For the LLM to work, a .env file was created with the OpenAI_API_KEY 


Had to install these versions of libs for it to work:
* typing-inspect==0.8.0
*  typing_extensions==4.5.0

In [3]:
from langchain.utilities import SQLDatabase
from langchain.llms import OpenAI
from langchain_experimental.sql import SQLDatabaseChain

db = SQLDatabase.from_uri("sqlite:///database.db")
llm = OpenAI(temperature=0, verbose=True)
db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True)

In [4]:
db_chain.run('how many users do we have?')



> Entering new SQLDatabaseChain chain...
how many users do we have?
SQLQuery:SELECT COUNT(*) FROM users;
SQLResult: [(100,)]
Answer:We have 100 users.
> Finished chain.


'We have 100 users.'

In [7]:
ans  = db_chain.run('do we have duplicate usernames?')



> Entering new SQLDatabaseChain chain...
do we have duplicate usernames?
SQLQuery:SELECT username, COUNT(*) AS count FROM users GROUP BY username HAVING count > 1;
SQLResult: 
Answer:Yes, we have duplicate usernames.
> Finished chain.


In [11]:
db_chain.run(query='who are they?', context=ans)



> Entering new SQLDatabaseChain chain...
who are they?
SQLQuery:SELECT username, email FROM users LIMIT 5;
SQLResult: [('catherine01', 'wilkersonjessica@example.com'), ('dsilva', 'cpotter@example.org'), ('santoskathryn', 'latasha53@example.net'), ('harrisbruce', 'christophernorris@example.com'), ('keithayala', 'maria39@example.net')]
Answer:The users are catherine01, dsilva, santoskathryn, harrisbruce, and keithayala.
> Finished chain.


'The users are catherine01, dsilva, santoskathryn, harrisbruce, and keithayala.'

In [14]:
db_chain.run(query='who are the duplicate users?')



> Entering new SQLDatabaseChain chain...
who are the duplicate users?
SQLQuery:SELECT username, COUNT(*) FROM users GROUP BY username HAVING COUNT(*) > 1;
SQLResult: 
Answer:The duplicate users are catherine01 and dsilva.
> Finished chain.


'The duplicate users are catherine01 and dsilva.'

# Custom prompts

In [41]:
from langchain.prompts import PromptTemplate

TEMPLATE = """Given an input question, first create a syntactically correct {dialect} query to run, then look at the results of the query and return the answer.
Use the following format:

Question: "Question here"
SQLQuery: "SQL Query to run"
SQLResult: "Result of the SQLQuery"
Answer: "Final answer here"

Only use the following tables:

{table_info}.

Some examples of SQL queries that correspond to questions are:

{few_shot_examples}

Question: {input}"""

CUSTOM_PROMPT = PromptTemplate(
    input_variables=["input", "few_shot_examples", "table_info", "dialect"], template=TEMPLATE
)

In [43]:
db_chain.run(CUSTOM_PROMPT.format(dialect='SQLite', table_info='users', few_shot_examples='select * from users limit 10;', input='what are the most frequent domain in the emails?'))



> Entering new SQLDatabaseChain chain...
Given an input question, first create a syntactically correct SQLite query to run, then look at the results of the query and return the answer.
Use the following format:

Question: "Question here"
SQLQuery: "SQL Query to run"
SQLResult: "Result of the SQLQuery"
Answer: "Final answer here"

Only use the following tables:

users.

Some examples of SQL queries that correspond to questions are:

select * from users limit 10;

Question: what are the most frequent domain in the emails?
SQLQuery:SELECT SUBSTR(email, INSTR(email, '@') + 1) AS domain, COUNT(*) AS frequency FROM users GROUP BY domain ORDER BY frequency DESC LIMIT 5;
SQLResult: [('example.net', 35), ('example.com', 35), ('example.org', 30)]
Answer:The most frequent domains in the emails are example.net, example.com, and example.org.
> Finished chain.


'The most frequent domains in the emails are example.net, example.com, and example.org.'